#

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

# Lexer

# Parser

In [52]:

@dataclass
class DimensionNode:
    value: any

@dataclass
class NumberNode:
    value: any

    def __repr__(self):
        return f"{self.value}"

@dataclass
class AddNode:
    node_a: any
    node_b: any

    def __repr__(self):
        return f"({self.node_a}+{self.node_b})"

@dataclass
class SubtractNode:
    node_a: any
    node_b: any

    def __repr__(self):
        return f"({self.node_a}-{self.node_b})"

@dataclass
class MultiplyNode:
    node_a: any
    node_b: any

    def __repr__(self):
        return f"({self.node_a}*{self.node_b})"

@dataclass
class DivideNode:
    node_a: any
    node_b: any

    def __repr__(self):
        return f"({self.node_a}/{self.node_b})"

@dataclass
class PowNode:
    node_a: any
    node_b: any
    def __repr__(self):
        return f"({self.node_a}**{self.node_b})"
    
@dataclass
class PlusNode:
    node: any

    def __repr__(self):
        return f"(+{self.node})"
    
@dataclass
class MinusNode:
    node: any

    def __repr__(self):
        return f"(-{self.node})"
    
class Parser():
    def __init__(self, tokens):
        # A parser is an iterable of tokens
        self.tokens = iter(tokens)
        self.advance()
        
    def raise_error(self):
        raise Exception("Invalid Syntax.")
        
    def advance(self):
        try:
            self.current_token = next(self.tokens)
        except StopIteration:
            self.current_token = None
            
    def parse(self):
        if self.current_token == None:
            return None
        result = self.expr()
        if self.current_token != None:
            self.raise_error()
        
        return result
    
    def expr(self):
        result = self.term()

        while self.current_token != None and self.current_token.type in (TokenType.PLUS,
                                                                         TokenType.MINUS):
            if self.current_token.type == TokenType.PLUS:
                self.advance()
                result = AddNode(result, self.term())
            elif self.current_token.type == TokenType.MINUS:
                self.advance()
                result = SubtractNode(result, self.term())
        return result
    

    def term(self):
        result = self.expo()

        while self.current_token != None and self.current_token.type in (TokenType.MULTIPLY,
                                                                         TokenType.DIVIDE):
            if self.current_token.type == TokenType.MULTIPLY:
                self.advance()
                result = MultiplyNode(result, self.expo())
            elif self.current_token.type == TokenType.DIVIDE:
                self.advance()
                result = DivideNode(result, self.expo())
                
        return result

    def expo(self):
        result = self.factor()
    
        while self.current_token != None and self.current_token.type == TokenType.POW:
            self.advance()
            result = PowNode(result, self.factor())    
        return result
    
    
    def factor(self):
        token = self.current_token

        if token.type == TokenType.LPAREN:
            self.advance()
            result = self.expr()

            if self.current_token.type != TokenType.RPAREN:
                self.raise_error()
            
            self.advance()
            return result

        
        elif token.type == TokenType.NUMBER:
            self.advance()
            return NumberNode(token.value)
        
        elif token.type == TokenType.DIMENSION:
            self.advance()
            return DimensionNode(token.value) 

        elif token.type == TokenType.PLUS:
            self.advance()
            return PlusNode(self.factor())
        
        elif token.type == TokenType.MINUS:
            self.advance()
            return MinusNode(self.factor())
        
        self.raise_error()

In [53]:
for text in to_parse:
    print(text)
    lexer = Lexer(text)
    lexed = list(lexer.generate_tokens())
    print(lexed)
    parser = Parser(lexed)
    tree = parser.parse()
    print(tree)

m
[DIMENSION:m]
DimensionNode(value='m')
L
[DIMENSION:L]
DimensionNode(value='L')
L*M/T**2
[DIMENSION:L, MULTIPLY, DIMENSION:M, DIVIDE, DIMENSION:T, POW, NUMBER:2]
((DimensionNode(value='L')*DimensionNode(value='M'))/(DimensionNode(value='T')**2))
L/T**2
[DIMENSION:L, DIVIDE, DIMENSION:T, POW, NUMBER:2]
(DimensionNode(value='L')/(DimensionNode(value='T')**2))
T**(-2)
[DIMENSION:T, POW, LPAREN, MINUS, NUMBER:2, RPAREN]
(DimensionNode(value='T')**(-2))
RAD
[DIMENSION:RAD]
DimensionNode(value='RAD')
L**2*T
[DIMENSION:L, POW, NUMBER:2, MULTIPLY, DIMENSION:T]
((DimensionNode(value='L')**2)*DimensionNode(value='T'))
L**2*T
[DIMENSION:L, POW, NUMBER:2, MULTIPLY, DIMENSION:T]
((DimensionNode(value='L')**2)*DimensionNode(value='T'))
L*M
[DIMENSION:L, MULTIPLY, DIMENSION:M]
(DimensionNode(value='L')*DimensionNode(value='M'))
theta
[DIMENSION:theta]
DimensionNode(value='theta')
L**10
[DIMENSION:L, POW, NUMBER:10]
(DimensionNode(value='L')**10)
I**2*T**3/(L**2*M)
[DIMENSION:I, POW, NUMBER:2, MULTI

In [54]:
text = "T**2.0*L"#**3/(L**2*M)"
lexer = Lexer(text)
lexed = list(lexer.generate_tokens())
print(lexed)
parser = Parser(lexed)
tree = parser.parse()
print(tree)

[DIMENSION:T, POW, NUMBER:2.0, MULTIPLY, DIMENSION:L]
((DimensionNode(value='T')**2.0)*DimensionNode(value='L'))


# Interpreter

In [56]:

        

text = "T**2.0"#*L**3*T"#/(L**2*M)"
lexer = Lexer(text)
lexed = list(lexer.generate_tokens())
print(lexed)
parser = Parser(lexed)
tree = parser.parse()
print(tree)

print("####INTERPRETER")
interpreter = Interpreter(tree)
res = interpreter.parse_tree()
print(type(res), res)
res.value

[DIMENSION:T, POW, NUMBER:2.0]
(DimensionNode(value='T')**2.0)
####INTERPRETER


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'float'

In [44]:
old = Dimension(text)

In [45]:
%timeit Dimension(text)

1.22 ms ± 26.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [33]:
£

In [34]:
repr(old)

"<Dimension : {'L': 3, 'M': 0, 'T': 3, 'I': 0, 'theta': 0, 'N': 0, 'J': 0, 'RAD': 0, 'SR': 0}>"

In [35]:
repr(new)

"<Dimension : {'L': 3, 'M': 0, 'T': 3, 'I': 0, 'theta': 0, 'N': 0, 'J': 0, 'RAD': 0, 'SR': 0}>"

In [36]:
from physipy.quantity.dimension import parse_str_to_dic

In [38]:
print(text)
print(parse_str_to_dic(text))
print(compute_dim_(text))

T**2.0*L**3*T
{'L': 3, 'T': 3.00000000000000}
{'L': 3, 'T': 3}


In [39]:
%timeit parse_str_to_dic(text)
%timeit compute_dim_(text)

596 µs ± 3.41 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
85.2 µs ± 794 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [17]:
old == new

True

In [24]:
%%timeit
res = (Interpreter(Parser(list(Lexer(text).generate_tokens())).parse()).parse_tree()).value
for k, v in res.dim_dict.items():
    if int(v) == v:
        res.dim_dict[k]=int(v)

84.7 µs ± 31.1 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [25]:
from physipy import units
ms = units["ms"]
mus = units['mus']
(1.22*ms)/(84.7*mus)

14.403778040141676